<a href="https://colab.research.google.com/github/GeorgeM2000/CANE/blob/master/code/Automatic_Keyword_Extraction_for_Citation_Graphs_with_KeyBERT_and_KeyLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Libraries & Tools***

In [2]:
!pip install keybert

In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00


In [4]:
import nltk
import spacy
import numpy as np
import gc
import os
import openai

from keybert import KeyBERT
from keybert import KeyLLM
from keybert.llm import OpenAI
from keybert.llm import TextGeneration
from keybert.llm import LiteLLM
from torch import cuda, bfloat16
import transformers
import bitsandbytes

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
nltk.data.path.append("...")

# ***Abstracts Retrieval***

In [5]:
def extract_abstracts(file_path):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        abstracts = file.readlines()

   # Remove any leading or trailing whitespace characters from each line
    abstracts = [abstract.strip() for abstract in abstracts if abstract.strip()]

    # Track the number of abstracts
    num_abstracts = len(abstracts)

    return abstracts, num_abstracts

# Example usage
file_path = 'cora/data.txt'
abstracts, num_abstracts = extract_abstracts(file_path)

# Display the number of extracted abstracts
print(f'Number of extracted abstracts: {num_abstracts}')

Number of extracted abstracts: 2277


In [6]:
abstracts

['Several computer algorithms discovering patterns groups protein sequences use based fitting parameters statistical model group related sequences These include hidden Markov model HMM algorithms multiple sequence alignment MEME Gibbs sampler algorithms discovering motifs These algorithms sometimes prone producing models incorrect two patterns combined The statistical model produced situation convex combination weighted average two different models This paper presents solution problem convex combinations form heuristic based using extremely low variance Dirichlet mixture priors part statistical model This heuristic call megaprior heuristic increases strength ie decreases variance prior proportion size sequence dataset This causes column final model strongly resemble mean single component prior regardless size dataset We describe cause convex combination problem analyze mathematically motivate describe implementation megaprior heuristic show effectively eliminate problem convex combinat

In [7]:
total_words = 0
for a in abstracts:
    total_words += len(a.split())

print(f'Total words in abstracts: {total_words}')

Total words in abstracts: 205936


# ***KeyBERT & KeyLLM***

## Keyword & Keyphrase Methods

In [8]:
def save_keywords_to_files(lists_of_keywords, file_names):
    """
    Save the keywords from each list to a separate text file.

    :param lists_of_keywords: A list of lists, where each sublist contains keywords for an abstract
    :param file_names: A list of file names corresponding to each list
    """
    for keywords_list, file_name in zip(lists_of_keywords, file_names):
        with open(file_name, 'w') as file:
            for keywords in keywords_list:
                # Join the keywords for the abstract into a single string
                line = ' '.join(keywords)
                # Write the line to the file
                file.write(line + '\n')

In [9]:
def modify_keyword_list(abstract_keywords):
    new_abstract_keywords = []
    for keyphrases in abstract_keywords:
        new_abstract_keywords.append([keyphrase[0] for keyphrase in keyphrases])
    return new_abstract_keywords

In [10]:
T = 5

## KeyBERT

In [ ]:
kbert = KeyBERT(model='all-MiniLM-L6-v2')

In [ ]:
abstract_embeddings, word_embeddings = kbert.extract_embeddings(abstracts, keyphrase_ngram_range=(1,3), stop_words='english')

In [ ]:
print(f'{type(abstract_embeddings)} \n{type(word_embeddings)}')
print(f'\n{abstract_embeddings.shape} \n{word_embeddings.shape}')

<class 'numpy.ndarray'> 
<class 'numpy.ndarray'>

(2277, 384) 
(307075, 384)


In [ ]:
np.save('KeyBERT_Abstract_Embeddings_Unigram.npy', abstract_embeddings)
np.save('KeyBERT_Abstract_Word_Embeddings_Unigram.npy', word_embeddings)

In [ ]:
np.save('KeyBERT_Abstract_Embeddings_Trigram.npy', abstract_embeddings)
np.save('KeyBERT_Abstract_Word_Embeddings_Trigram.npy', word_embeddings)

### KeyBERT for keyword extraction

In [ ]:
keybert_kws = kbert.extract_keywords(abstracts, keyphrase_ngram_range=(1,1), stop_words='english',
                                       use_maxsum=True, nr_candidates=20, top_n=T,
                                       doc_embeddings=abstract_embeddings, word_embeddings=word_embeddings)

In [ ]:
len(keybert_kws)

2277

In [ ]:
modified_keybert_kws = modify_keyword_list(keybert_kws)

### KeyBERT for keyphrase extraction

In [ ]:
keybert_kphs = kbert.extract_keywords(abstracts, keyphrase_ngram_range=(1,3), stop_words='english',
                                       use_maxsum=True, nr_candidates=20, top_n=T,
                                       doc_embeddings=abstract_embeddings, word_embeddings=word_embeddings)

In [ ]:
len(keybert_kphs)

2277

In [ ]:
modified_keybert_kphs = modify_keyword_list(keybert_kphs)

## KeyLLM

In [8]:
prompt = """
I have the following document:
[DOCUMENT]

Based on the information above, extract five keywords that best describe the topic of the text.
The keywords should be separated by commas. Make sure you to only return the keywords and say nothing else.
"""

The cell below uses a LLM from OpenAI. OpenAI requires an API key. Usage for OpenAI LLMs is limited and thus not appropriate for the current task.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

llm = OpenAI(client=client, prompt=prompt)

kLLM = KeyLLM(llm)

In [7]:
model_id = 'bit-dny/MindLLM-1b3-chat-zh-v2.0'

# 4-bit Quantization to load Llama 2 with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Llama 2 Model & Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.32M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/507k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [10]:
llm = TextGeneration(generator, prompt=prompt)
kLLM = KeyLLM(llm)

### KeyLLM for keyword extraction

In [ ]:
kLLM_openai_kws = kLLM.extract_keywords(abstracts)

### KeyLLM for keyphrase extraction

## Create Keyword Text Files

In [ ]:
# Combine the lists and provide corresponding file names
lists_of_keywords = [modified_keybert_kws, modified_keybert_kphs]
file_names = ['cora/KeyBERT_Unigram.txt', 'cora/KeyBERT_Trigram.txt']

# Save the keywords to separate text files
save_keywords_to_files(lists_of_keywords, file_names)
